# Tableau Workshop Data Preparation

#### Workshop on July 20, 2020

This notebook documents how I downloaded and prepared the datasets used in the Tableau Public workshop. The resulting data is already available to you in the [data](https://github.com/DataCircles/projects_circle/tree/master/traffic_collisions_project/workshop_materials/data) folder.    

## Download Seattle Street Shapefiles

Go to https://data-seattlecitygis.opendata.arcgis.com/datasets/seattle-streets and download the shapefile like in this screenshot:  

<img src="images/seattle_st_shapefile.png" alt="download shapefile" width="900"/>  

<br>

The following cells download and format the traffic collision data.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 500
pd.options.display.max_columns = 80

import warnings
warnings.filterwarnings('ignore') 

from datetime import datetime

from IPython.display import display, Markdown

# Display all output within each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Get the Collision Data

In [2]:
url = "https://data-seattlecitygis.opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv"
df = pd.read_csv(url, parse_dates=['INCDTTM'])
print("original df shape:", df.shape)
display(df.head(3))

original df shape: (220436, 40)


,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.340472,47.608629,1,18600,18600,1785104,Matched,Intersection,29598.0,PIKE PL AND PIKE ST,NaN,NaN,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2004/10/14 00:00:00+00,2004-10-14 18:36:00,At Intersection (intersection related),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,0,Overcast,Dry,Dark - Street Lights On,NaN,4288030.0,NaN,3,Vehicle backing hits pedestrian,0,0,N
1,-122.251788,47.508176,2,328272,329772,EA07021,Unmatched,Block,NaN,S PRENTICE ST BETWEEN 65TH AVE S AND 66TH AVE S,NEI,"Not Enough Information, or Insufficient Locati...",1,Property Damage Only Collision,NaN,2,0,0,0,0,0,0,2020/01/22 00:00:00+00,2020-01-22 00:00:00,Mid-Block (not related to intersection),12.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,Y
2,-122.328526,47.703180,3,328374,329874,EA09347,Matched,Intersection,37555.0,1ST AVE NE AND NE 103RD ST,,NaN,1,Property Damage Only Collision,Angles,4,0,0,2,0,0,0,2020/01/05 00:00:00+00,2020-01-05 13:28:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Raining,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N


## Drop Non-Useful Data

I am dropping unnecessary columns and some rows with null values so Tableau Public can process the data faster and to help me when I visualize the data.

In [3]:
# making the CSV file smaller so it's easier for Tableau Public to process

print(df.shape)

drop_cols = ["OBJECTID", "STATUS", "INCKEY",
             "COLDETKEY", "INTKEY", "SDOTCOLNUM",
             "EXCEPTRSNCODE", "EXCEPTRSNDESC", "PEDROWNOTGRNT"]
for col in drop_cols:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)
    else:
        continue

df.dropna(subset=["X", "Y"], inplace=True)
df.dropna(subset=["COLLISIONTYPE"], inplace=True)

print(df.shape)
df.head()

(220436, 40)
(189028, 31)


,X,Y,REPORTNO,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.340472,47.608629,1785104,Intersection,PIKE PL AND PIKE ST,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2004/10/14 00:00:00+00,2004-10-14 18:36:00,At Intersection (intersection related),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,0,Overcast,Dry,Dark - Street Lights On,NaN,3,Vehicle backing hits pedestrian,0,0,N
2,-122.328526,47.703180,EA09347,Intersection,1ST AVE NE AND NE 103RD ST,1,Property Damage Only Collision,Angles,4,0,0,2,0,0,0,2020/01/05 00:00:00+00,2020-01-05 13:28:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Raining,Wet,Daylight,NaN,10,Entering at angle,0,0,N
3,-122.320383,47.539432,3857045,Intersection,ELLIS AVE S AND S MYRTLE ST,1,Property Damage Only Collision,Sideswipe,2,0,0,2,0,0,0,2020/02/13 00:00:00+00,2020-02-13 08:12:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Dry,Daylight,NaN,11,From same direction - both going straight - bo...,0,0,N
6,-122.346793,47.662069,EA05985,Block,N 46TH ST BETWEEN PHINNEY TURN RD AND GREEN LA...,1,Property Damage Only Collision,Other,2,0,0,2,0,0,0,2020/01/21 00:00:00+00,2020-01-21 08:48:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Overcast,Wet,Daylight,NaN,23,From same direction - all others,0,0,N
8,-122.318085,47.550433,1795676,Block,AIRPORT WAY S BETWEEN S HOMER ST AND S DORIS ST,1,Property Damage Only Collision,Other,2,0,0,2,0,0,0,2004/09/21 00:00:00+00,2004-09-21 10:34:00,Mid-Block (but intersection related),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Clear,Dry,Daylight,NaN,21,One car entering driveway access,0,0,N


## Reformat the Data

I am relabeling some variables and condensing the labels in other columns to help with plotting. Note - this can be done in Tableau too. 

In [4]:
categories = ["ADDRTYPE", "COLLISIONTYPE", "SEVERITYDESC", "LIGHTCOND",
             "SPEEDING", "JUNCTIONTYPE", "ROADCOND", "WEATHER"]

for col in categories:
    print("\n{}: {} unique and {} null".format(col,
                                               df[col].nunique(dropna=False),
                                               df[col].isna().sum()))
    df[col].unique()
    
    
# relabel null & "Other" as "Unknown"
for col in categories:
    df[col] = df[col].replace({np.nan: "Unknown", "Other": "Unknown"})
    
print("\nChecking...\n")

# doublecheck no more null values
for col in categories:
    print("\n{}: {} unique and {} null".format(col,
                                               df[col].nunique(dropna=False),
                                               df[col].isna().sum()))


ADDRTYPE: 2 unique and 0 null


array(['Intersection', 'Block'], dtype=object)


COLLISIONTYPE: 10 unique and 0 null


array(['Pedestrian', 'Angles', 'Sideswipe', 'Other', 'Left Turn',
       'Rear Ended', 'Parked Car', 'Cycles', 'Right Turn', 'Head On'],
      dtype=object)


SEVERITYDESC: 5 unique and 0 null


...


ROADCOND: 10 unique and 123 null


array(['Dry', 'Wet', 'Unknown', 'Ice', 'Sand/Mud/Dirt', 'Snow/Slush',
       'Standing Water', nan, 'Other', 'Oil'], dtype=object)


WEATHER: 12 unique and 201 null


array(['Overcast', 'Raining', 'Clear', 'Unknown', 'Snowing',
       'Fog/Smog/Smoke', nan, 'Other', 'Partly Cloudy',
       'Sleet/Hail/Freezing Rain', 'Blowing Sand/Dirt',
       'Severe Crosswind'], dtype=object)


ADDRTYPE: 2 unique and 0 null

COLLISIONTYPE: 10 unique and 0 null

SEVERITYDESC: 5 unique and 0 null

LIGHTCOND: 8 unique and 0 null

SPEEDING: 2 unique and 0 null

JUNCTIONTYPE: 7 unique and 0 null

ROADCOND: 8 unique and 0 null

WEATHER: 10 unique and 0 null


In [5]:
df["WEATHER"] = df["WEATHER"].replace({"Partly Cloudy": "Overcast", np.nan: "Unknown",
                                       "Other": "Unknown", "Blowing Sand/Dirt": "Windy",
                                       "Severe Crosswind": "Windy", "Snowing": "Snowing/Hailing",
                                       "Sleet/Hail/Freezing Rain": "Snowing/Hailing"})

df["UNDERINFL"] = df["UNDERINFL"].replace({"0": "N", "1": "Y"})

I am adding the year, month, and time columns to make it easier to filter the data. You can also do this in Tableau if you want.  

In [6]:
df["INCDATE"] = df["INCDATE"].astype("datetime64")
df["YEAR"] = pd.DatetimeIndex(df["INCDATE"]).year 
df["MONTH"] = df["INCDATE"].dt.month_name()
df["COLLISION_TIME"] = df["INCDTTM"].dt.time
# df["HOUR"] = df["INCDTTM"].dt.hour  # if you just want the hour

In [7]:
# remove 2003 data (if any left)
df = df[df["YEAR"] != "2003"]

# final dataset
df.shape
df.head()

(189028, 34)

,X,Y,REPORTNO,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,YEAR,MONTH,COLLISION_TIME
0,-122.340472,47.608629,1785104,Intersection,PIKE PL AND PIKE ST,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2004-10-14,2004-10-14 18:36:00,At Intersection (intersection related),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,N,Overcast,Dry,Dark - Street Lights On,Unknown,3,Vehicle backing hits pedestrian,0,0,N,2004,October,18:36:00
2,-122.328526,47.703180,EA09347,Intersection,1ST AVE NE AND NE 103RD ST,1,Property Damage Only Collision,Angles,4,0,0,2,0,0,0,2020-01-05,2020-01-05 13:28:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Raining,Wet,Daylight,Unknown,10,Entering at angle,0,0,N,2020,January,13:28:00
3,-122.320383,47.539432,3857045,Intersection,ELLIS AVE S AND S MYRTLE ST,1,Property Damage Only Collision,Sideswipe,2,0,0,2,0,0,0,2020-02-13,2020-02-13 08:12:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Dry,Daylight,Unknown,11,From same direction - both going straight - bo...,0,0,N,2020,February,08:12:00
6,-122.346793,47.662069,EA05985,Block,N 46TH ST BETWEEN PHINNEY TURN RD AND GREEN LA...,1,Property Damage Only Collision,Unknown,2,0,0,2,0,0,0,2020-01-21,2020-01-21 08:48:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Overcast,Wet,Daylight,Unknown,23,From same direction - all others,0,0,N,2020,January,08:48:00
8,-122.318085,47.550433,1795676,Block,AIRPORT WAY S BETWEEN S HOMER ST AND S DORIS ST,1,Property Damage Only Collision,Unknown,2,0,0,2,0,0,0,2004-09-21,2004-09-21 10:34:00,Mid-Block (but intersection related),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Clear,Dry,Daylight,Unknown,21,One car entering driveway access,0,0,N,2004,September,10:34:00


## Save the Data

Save the CSV file so it can be uploaded to Tableau Public. 

In [8]:
# to save the CSV file data locally in your working directory:

now = datetime.now()
file_name = "collision_data_{}{}{}.csv".format(now.month, now.day, now.year)
df.to_csv(file_name, encoding="utf-8", index=False)